# Torch common examples

In [ ]:
import copy
import pickle

import numpy as np
from datasets import load_dataset
from plaid.bridges.huggingface_bridge import (
    huggingface_dataset_to_plaid,
    huggingface_description_to_problem_definition,
)
from plaid.containers.sample import Sample
from torch.utils.data import DataLoader

from plaid_bridges.common.base_regression_dataset import BaseRegressionDataset
from plaid_bridges.common.homogeneous_dataset import HomogeneousDataset

In [ ]:
hf_dataset = load_dataset("PLAID-datasets/VKI-LS59", split="all_samples[:10]")
sample = Sample.model_validate(pickle.loads(hf_dataset[0]["sample"]))

pb_def = huggingface_description_to_problem_definition(hf_dataset.info.description)
ids = pb_def.get_split("train")[:10]

dataset, _ = huggingface_dataset_to_plaid(hf_dataset, ids=ids, processes_number=5)

In [ ]:
print(dataset)
all_feat_ids = dataset[0].get_all_features_identifiers()

scalar_features = [f for f in all_feat_ids if "scalar" in f.values()]
field_features = [f for f in all_feat_ids if "field" in f.values()]

in_features_identifiers = [scalar_features[0], field_features[0]]
out_features_identifiers = [field_features[1], scalar_features[1]]

print(in_features_identifiers)
print(out_features_identifiers)

In [ ]:
reg_dataset = BaseRegressionDataset(
    dataset=dataset,
    in_features_identifiers=in_features_identifiers,
    out_features_identifiers=out_features_identifiers,
    train=True,
)

reg_dataset.show_details()

In [ ]:
print(reg_dataset[1][0][1])
print(reg_dataset.in_features[1][1])

## Homogeneous dataset

In [ ]:
scalar_features = [f for f in all_feat_ids if "scalar" in f.values()]
field_features = [
    f for f in all_feat_ids if "field" in f.values() and "Base_2_2" in f.values()
]

in_features_identifiers = scalar_features
out_features_identifiers = field_features

print(in_features_identifiers)
print(out_features_identifiers)

In [ ]:
homogen_dataset = HomogeneousDataset(
    dataset=dataset,
    in_features_identifiers=in_features_identifiers,
    out_features_identifiers=out_features_identifiers,
    train=True,
)

homogen_dataset.show_details()

In [ ]:
loader = DataLoader(
    homogen_dataset,
    batch_size=2,
    shuffle=False,
)

out_feat_id = out_features_identifiers[0]
before = copy.deepcopy(dataset[ids[1]].get_feature_from_identifier(out_feat_id))

predictions = []
for batch in loader:
    for torch_sample in batch:
        predictions.append(torch_sample[1])

dataset_pred = homogen_dataset.inverse_transform(predictions)

after = copy.deepcopy(dataset_pred[ids[1]].get_feature_from_identifier(out_feat_id))

print("Error after transform then inverse transform (2nd sample):")
print(np.linalg.norm(after - before) / np.linalg.norm(before))